In [0]:
import os
import shutil
import boto3
from botocore import UNSIGNED
from botocore.client import Config
from tqdm import tqdm
from multiprocessing import Pool

In [0]:
def download_s3(page):
    for obj in page.get('Contents', []):
        key = obj['Key']
        filename = key.split('/')[-1]
        subfolder_path_temp = os.path.join(temp_dir, os.path.dirname(key[len(s3_folder):]))
        subfolder_path = os.path.join(local_dir, os.path.dirname(key[len(s3_folder):]))
        os.makedirs(subfolder_path_temp, exist_ok=True)
        os.makedirs(subfolder_path, exist_ok=True)
        local_file_path_temp = os.path.join(subfolder_path_temp, filename)
        local_file_path = os.path.join(subfolder_path, filename)
        if filename:
            s3.download_file(bucket_name, key, local_file_path_temp)
            shutil.copy2(local_file_path_temp,local_file_path)
            os.remove(local_file_path_temp)
            
def download_s3_folder(bucket_name, s3_folder):
    global s3
    s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))
    if not s3_folder.endswith('/'):
        s3_folder += '/'
    os.makedirs(local_dir, exist_ok=True)
    paginator = s3.get_paginator("list_objects_v2").paginate(Bucket=bucket_name, Prefix=s3_folder)
    with Pool(processes= 20) as p:
        results = p.imap(download_s3, paginator)
        tuple(results)

In [0]:
bucket_name = 'surftemp-sst'
s3_folder = 'data/'
temp_dir = '/tmp/usp/ProjetoIntegrador/rawzone/surftemp-sst/'
local_dir = '/dbfs/mnt/usp/ProjetoIntegrador/rawzone/surftemp-sst/'

paginator = download_s3_folder(bucket_name, s3_folder)